<a href="https://colab.research.google.com/github/BaronVonBussin/NewTransit/blob/main/get_ticker_data_20241223.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import yfinance as yf

# List of tickers
tickers = ["SPY", "XLE", "XLI", "XLK", "XLU", "XLF", "XLP", "XLY", "XLV", "XLB"]

# Download data since 1995
data = yf.download(tickers, start="1995-01-01", end="2024-12-19", group_by="ticker")

# Save each ticker's data as a CSV
for ticker in tickers:
    data[ticker].to_csv(f"{ticker}_OHLC.csv")

print("OHLC data downloaded and saved.")


[*********************100%***********************]  10 of 10 completed


OHLC data downloaded and saved.


In [5]:
import pandas as pd

# Load data from CSV
def load_data(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)
    # Ensure 'Date' column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    # Set 'Date' as the index
    df.set_index('Date', inplace=True)
    return df

# Aggregate data to weekly and monthly levels
def aggregate_data(df):
    # Weekly aggregation (sum)
    weekly_data = df.resample('W').sum()

    # Monthly aggregation (mean)
    monthly_data = df.resample('ME').mean()

    return weekly_data, monthly_data

# Save aggregated data to CSV
def save_to_csv(data, file_name):
    data.to_csv(file_name)
    print(f"Saved {file_name}")

# Main program
if __name__ == "__main__":
    # Path to the input CSV file
    input_file = "/content/XLB_OHLC.csv"  # Replace with your file path

    print("Loading data from CSV...")
    df = load_data(input_file)
    print("Daily Data:")
    print(df.head())

    print("\nAggregating data to weekly and monthly levels...")
    weekly_data, monthly_data = aggregate_data(df)

    print("\nWeekly Data (Sum):")
    print(weekly_data.head())

    print("\nMonthly Data (Mean):")
    print(monthly_data.head())

    # Save aggregated data to CSV files
    save_to_csv(weekly_data, "XLB_weekly_data.csv")
    save_to_csv(monthly_data, "XLB_monthly_data.csv")
    print("\nAggregated data saved to CSV files.")


Loading data from CSV...
Daily Data:
            Open  High  Low  Close  Adj Close  Volume
Date                                                 
1995-01-03   NaN   NaN  NaN    NaN        NaN     NaN
1995-01-04   NaN   NaN  NaN    NaN        NaN     NaN
1995-01-05   NaN   NaN  NaN    NaN        NaN     NaN
1995-01-06   NaN   NaN  NaN    NaN        NaN     NaN
1995-01-09   NaN   NaN  NaN    NaN        NaN     NaN

Aggregating data to weekly and monthly levels...

Weekly Data (Sum):
            Open  High  Low  Close  Adj Close  Volume
Date                                                 
1995-01-08   0.0   0.0  0.0    0.0        0.0     0.0
1995-01-15   0.0   0.0  0.0    0.0        0.0     0.0
1995-01-22   0.0   0.0  0.0    0.0        0.0     0.0
1995-01-29   0.0   0.0  0.0    0.0        0.0     0.0
1995-02-05   0.0   0.0  0.0    0.0        0.0     0.0

Monthly Data (Mean):
            Open  High  Low  Close  Adj Close  Volume
Date                                                 
1995-01